<a href="https://colab.research.google.com/github/Golem8/Music-Genre-Classifier/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Imports
import os
!pip install pydub
from pydub.utils import make_chunks
from pydub import AudioSegment
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import librosa

In [3]:
# this could differ based on how you saved the shared folder
#sharedPath = r'/content/drive/MyDrive/APS360 Team Project' # Simon's path
sharedPath = r'/content/drive/MyDrive/APS360 Team Project' # Daniels path
#sharedPath = r'/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project' # Matthew's path

In [7]:
smallSet = True

for block in ['block1','block2','block3','block4','block5']:
  if smallSet:
    blockPath = os.path.join(sharedPath, 'data', 'small_baseline', block)
  else:
    blockPath = os.path.join(sharedPath, 'data', block)
  for filename in os.listdir(blockPath):
    if filename.endswith('.wav'):
      twoSecClip = AudioSegment.from_wav(os.path.join(blockPath,filename))

      #each subclip is 200ms long
      chunks = make_chunks(twoSecClip, 200)

      # iterate over the newly created objects
      for chunk in chunks:
        if len(chunk) != 200:
          print('Something went wrong with chunk splitting')
        else:

          # make it mono instead of stero for fourier transform 
          chunk = chunk.set_channels(1)
          
          # this makes an array of samples
          # librosa loads fils as float32, so convert it
          # https://stackoverflow.com/questions/58810035/converting-audio-files-between-pydub-and-librosa
          chunk_array = np.array(chunk.get_array_of_samples()).astype(np.float32) # /32768

          # kaiser_best is high quality mode, both the original and trarget sr are the same
          chunk_array = librosa.core.resample(chunk_array, chunk.frame_rate, chunk.frame_rate, res_type='kaiser_best') 

          x = chunk_array
          sr = chunk.frame_rate

          y = np.fft.fft(x)
          fr_xaxis_range = np.array(range(0, len(y))) / 4

          #plt.plot(fr_xaxis_range, np.abs(y))
          #plt.xlabel('Frequency in Hertz')
          print(y)


[ 714869.             +0.j         1028860.4545602 -283322.89611752j
 -648183.91542141-278186.53147593j ... -804118.7883993 +921588.97168207j
 -648183.91542141+278186.53147593j 1028860.4545602 +283322.89611752j]
[ -661133.             +0.j         -1543416.18695267-562399.14044367j
 -1578458.62314848-319359.46526791j ... -2064331.2727737 -608393.19881941j
 -1578458.62314848+319359.46526791j -1543416.18695267+562399.14044367j]
[1448371.              +0.j         2198755.03124243-1425407.03447099j
 3677320.20047277-1340273.95467184j ... 3799584.21516709 +297233.15165122j
 3677320.20047277+1340273.95467184j 2198755.03124243+1425407.03447099j]
[ -994101.              +0.j         -4230456.38238409 -345081.50795002j
 -2022552.79155986  +45520.91014249j ...
 -4223894.80567735+1063543.09273587j -2022552.79155986  -45520.91014249j
 -4230456.38238409 +345081.50795002j]
[  46667.             +0.j           -8696.20255184+949958.17730759j
  384471.16995969+414941.45899175j ... -207135.79953303-34